In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F

In [10]:
def convblock(in_channels, out_channels,kernel_size=3,stride=1,padding=1,pool=False):
    if pool:
        return nn.Sequential(
            nn.Conv2d(in_channels,out_channels, kernel_size=kernel_size, stride=stride,padding=padding),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.BatchNorm2d(num_features=out_channels),
            nn.ReLU(),
        )
    else:
        return nn.Sequential(
            nn.Conv2d(in_channels,out_channels, kernel_size=kernel_size, stride=stride,padding=padding),
            nn.BatchNorm2d(num_features=out_channels),
            nn.ReLU(),
    )


In [5]:
def linearblock(in_features, out_features):
    return nn.Sequential(
        nn.Linear(in_features=in_features, out_features=out_features),
        nn.ReLU(),
        nn.Dropout(p=0.5)
    )

In [6]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.block1 = convblock(3,96,11,4,0,pool=True)
        self.block2 = convblock(96,256,5,1,2,pool=True)
        self.block3 = convblock(256,384,3,1,1)
        self.block4 = convblock(384,384,3,1,1)
        self.block5 = convblock(384,256,3,1,1)
        self.pool = nn.AdaptiveAvgPool2d((6,6))
        self.fc1 = linearblock(6*6*256,4096)
        self.fc2 = linearblock(4096, 4096)
        self.fc3 = nn.Linear(in_features=4096,out_features=1000)
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.pool(x)
        x = x.view(-1, 6*6*256)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


        

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
model = AlexNet()
x = torch.randn(1,3,224,224)

In [28]:
output = model(x)

In [29]:
output.shape

torch.Size([1, 1000])